# AutoGEN 第六课： 避一个坑

这节内容是对之前几节的一个补充，在前面我们提到user proxy agent具有执行代码的能力，换一个思路，它的能力是不是有点太强了，会不会失控对宿主机电脑造成伤害呢？ 
答案显然是，可以造成伤害的。即便我们可以通过人工干预的方式，降低它的风险，但是不能保证就一定能避免这个问题。
因此这里会引入一个docker容器的东西，简单理解，就是创建了一个简化版本的虚拟环境，在那个里面瞎搞都不会影响外面宿主机的环境。

我们通过下面的一个小案例来了解一下：
我们不使用docker，然后给一个agent发送一个删除当前目录下文件的命令，看看会发生什么？


## 1. 配置环境

1. 你需要安装autogent的库， 要求python版本>=3.8
2. 配置openai 或者 azure openai的api key, 并将key写入到一个名为 "OAI_CONFIG_LIST"的文件中，并将该文件放在项目目录下。或者将其配置到你的环境变量中。

OAI_CONFIG_LIST的内容结构如下：
```python
[
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]

In [1]:
%pip install pyautogen~=0.1.0 -q

Note: you may need to restart the kernel to use updated packages.


## 2.不使用docker

In [1]:
# 在这里，我们首先手动创建一个名为 FILE_TO_REMOVE.txt的文件

file_path = 'FILE_TO_REMOVE.txt'
file = open(file_path, 'w')

# 在文件中写入内容
file.write('Hello, this is an example file.')

# 关闭文件
file.close()

In [3]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "config_list": config_list,  
        "temperature": 0,  
    },  
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE", 
    max_consecutive_auto_reply=10,      
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"), 
    code_execution_config={
        "work_dir": "coding",   
        "use_docker": False,    
    }
)

user_proxy.initiate_chat(
    assistant,
    message="""帮我查看当前目录下有没有一个名为 "FILE_TO_REMOVE.txt" 的文件，如果有就删除它，如何没有就说没有并结束任务""",
)

user_proxy (to assistant):

帮我查看当前目录下有没有一个名为 "FILE_TO_REMOVE.txt" 的文件，如果有就删除它，如何没有就说没有并结束任务

--------------------------------------------------------------------------------
assistant (to user_proxy):

首先，我们需要使用Python的os模块来检查文件是否存在，然后删除它。以下是可以执行的Python代码：

```python
import os

# 指定要检查的文件名
filename = "FILE_TO_REMOVE.txt"

# 检查文件是否存在
if os.path.exists(filename):
    # 如果存在，删除它
    os.remove(filename)
    print(f"'{filename}' 文件已被删除.")
else:
    # 如果不存在，打印消息并结束任务
    print(f"'{filename}' 文件不存在.")
```

请将上述代码复制并在Python环境中执行。

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
'FILE_TO_REMOVE.txt' 文件不存在.


--------------------------------------------------------------------------------
assistant (to user_proxy):

根据执行结果，"FILE_TO_REMOVE.txt" 文件不存在。这就是我们需要的结果，任务已经完成。

TERMINATE

---------------

虽然有一点小波折，但最后你会发现这个文件确实被删除了，因此在对待一些特殊的任务的时候，谨慎选择。

另外，如何选择使用docker呢？
1. 首先你需要在本机安装docker客户端，并且让docker可以被命令行使用，根据系统不同window/macos/linux等等，解决方法也不一样，it's really a long history! 自己问gpt吧。

2. 然后安装docker的python库， pip install docker

3. 注释掉这段代码就可以了， 默认会启用docker的。



In [5]:
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE", 
    max_consecutive_auto_reply=10,      
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"), 
    code_execution_config={
        "work_dir": "coding",   
        # "use_docker": False
    }
)